### Load library

In [ ]:
library(TwoSampleMR)
library(data.table)
library(R.utils)

### form exposure data

In [ ]:
# Read raw data and filter by pvalue

# cc = as.data.frame(fread("/course/bgi23/huanhuan/case-control-2.txt",header = T))
wh = as.data.frame(fread("/course/bgi23/huanhuan/EOS_pct.ss",header = T))
wh_sig = wh[wh$pvalue < 5e-08 & wh$SNP != ".",]

In [ ]:
# format data
exposure_dat = format_data(
  wh_sig,
  type = "exposure",
  phenotype_col = "trait",
  snp_col = "SNP",
  beta_col = "beta",
  se_col = "se",
  eaf_col = "eaf",
  effect_allele_col = "effect_allele",
  other_allele_col = "other_allele",
  pval_col = "pvalue",
  samplesize_col = "samplesize"
)
# exposure_dat$chr.exposure = gsub("chr","",exposure_dat$chr.exposure)

In [ ]:
# Clumping data
exp_dat = clump_data(
  exposure_dat,
  clump_kb = 10000,
  clump_r2 = 0.1,
  clump_p1 = 1,
  clump_p2 = 1,
  pop = "EAS"
)


### form outcome data

In [ ]:
############################ outcome variable ########
trait = "Prostate_cancer"
ncase = 5408
ncontrol = 103939

In [ ]:
# read raw data and do filtering
bbj = as.data.frame(fread("/course/bgi23/huanhuan/phenocode-Prostate_cancer.tsv.gz",header = T))
bbj_slt = bbj[bbj$rsids %in% exp_dat$SNP,]
bbj_slt$trait = trait
bbj_slt$ncase = ncase
bbj_slt$ncontrol = ncontrol

In [ ]:
# format data
out_dat = format_data(
  bbj_slt,
  type = "outcome",
  phenotype_col = "trait",
  snp_col = "rsids",
  beta_col = "beta",
  se_col = "sebeta",
  eaf_col = "maf",
  effect_allele_col = "alt",
  other_allele_col = "ref",
  pval_col = "pval",
  ncase_col = "ncase",
  ncontrol_col = "ncontrol",
)

### Harmonize data

In [ ]:
dat <- harmonise_data(exp_dat, out_dat)

### run MR analysis

In [ ]:
res <- mr(dat)

### view the result

In [ ]:
res